In [4]:
import yfinance as yf

In [5]:
start_date = '2013-01-01'
end_date = '2023-01-01'


# Scarica i dati per jonson and jonson:
jnj = yf.download('jnj', start=start_date, end=end_date)

# Scarica i dati per adobe :
adbe = yf.download('adbe', start=start_date, end=end_date)

# Funzione per creare tabelle annuali con prezzi di chiusura
def create_annual_close_table(data, asset_name):
    # Rimuovi le osservazioni con valori nulli o NaN
    data = data.dropna()
    years = data.index.year.unique()
    annual_data = []

    for year in years:
        yearly_data = data[data.index.year == year]
        close_prices = yearly_data['Close'].tolist()
        annual_data.append([year] + close_prices)

    annual_df = pd.DataFrame(annual_data)
    annual_df.columns = ['Year'] + [f'Day_{i+1}' for i in range(len(annual_df.columns) - 1)]
    return annual_df

# Creazione delle tabelle annuali per i prezzi di chiusura
jnj_close_annual = create_annual_close_table(jnj, 'adbe')
adbe_close_annual = create_annual_close_table(adbe, 'jnj')

# Esporta le tabelle annuali in formato CSV
jnj_close_annual.to_csv('/Users/roberto/Desktop/UNI 3^ ANNO/tesi/dataset/data_jnj.csv', index=False)
adbe_close_annual.to_csv('/Users/roberto/Desktop/UNI 3^ ANNO/tesi/dataset/data_adbe.csv', index=False)

print("Tabelle annuali dei prezzi di chiusura create e salvate in file CSV separati con successo!")




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


AttributeError: 'DataFrame' object has no attribute 'tolist'

In [16]:
import pandas as pd

# Carica il dataset dal file CSV
csv_file = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/dataset/data_jnj.csv'  # Inserisci il percorso del tuo file CSV
dati = pd.read_csv(csv_file)

# Elimina le ultime due colonne
dati_senza_ultime_due = dati.iloc[:, :-2]

# Salva il dataset modificato in un nuovo file CSV
dati_senza_ultime_due.to_csv('/Users/roberto/Desktop/UNI 3^ ANNO/tesi/dataset/adbe.csv', index=False)

print("Le ultime due colonne sono state eliminate con successo.")


Le ultime due colonne sono state eliminate con successo.


In [17]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

def carica_dati(csv_file):
    """
    Carica i dati di serie storiche da un file CSV.
    Supponiamo che ci sia una colonna 'Date' e una colonna 'Close' con i prezzi di chiusura.
    """
    dati = pd.read_csv(csv_file)
    dati['Date'] = pd.to_datetime(dati['Date'])
    dati['Year'] = dati['Date'].dt.year
    return dati

def genera_rumore_bianco(seg_data, snr_db=20):
    """
    Aggiunge rumore bianco gaussiano ai dati di chiusura con un rapporto segnale-rumore specificato.
    """
    potenza_segnale = np.mean(seg_data**2)
    potenza_rumore = potenza_segnale / (10**(snr_db / 10))
    rumore_bianco = np.sqrt(potenza_rumore) * np.random.randn(len(seg_data))
    dati_con_rumore = seg_data + rumore_bianco
    return dati_con_rumore

def genera_spettrogrammi_per_anno(dati, anno, num_spettrogrammi, output_dir, snr_db=20):
    """
    Genera 200 spettrogrammi per un dato anno e li salva nella cartella di output.
    """
    # Filtra i dati per l'anno specificato
    dati_anno = dati[dati['Year'] == anno]['Close'].values
    
    # Dividi i dati in segmenti per generare più spettrogrammi
    segment_length = len(dati_anno) // num_spettrogrammi
    
    for i in range(num_spettrogrammi):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Seleziona i dati del segmento
        seg_data = dati_anno[start_idx:end_idx]
        
        # Aggiungi rumore bianco
        seg_data_rumore = genera_rumore_bianco(seg_data, snr_db)
        
        # Genera lo spettrogramma
        f, t, Sxx = signal.spectrogram(seg_data_rumore, nperseg=128)
        
        # Plot dello spettrogramma
        plt.figure(figsize=(5, 4))
        plt.pcolormesh(t, f, 10 * np.log10(Sxx))
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [sec]')
        plt.title(f'Spettrogramma - Anno {anno} - {i+1}')
        
        # Salva lo spettrogramma
        file_name = f'spettrogramma_anno_{anno}_{i+1}.png'
        output_path = os.path.join(output_dir, file_name)
        plt.savefig(output_path)
        plt.close()

def genera_spettrogrammi_per_tutti_gli_anni(csv_file, output_dir, num_spettrogrammi_per_anno=200, snr_db=20):
    """
    Genera spettrogrammi per ogni anno nel file CSV e li salva nella cartella di output.
    """
    # Carica i dati dal file CSV
    dati = carica_dati(csv_file)
    
    # Crea la cartella di output se non esiste
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Trova gli anni unici
    anni = dati['Year'].unique()
    
    # Genera gli spettrogrammi per ogni anno
    for anno in anni:
        genera_spettrogrammi_per_anno(dati, anno, num_spettrogrammi_per_anno, output_dir, snr_db)

# Esempio di utilizzo
csv_file = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/dataset/adbe.csv'  # Percorso del file CSV con i dati
output_dir = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectograms_adbe'  # Cartella di output per salvare gli spettrogrammi
genera_spettrogrammi_per_tutti_gli_anni(csv_file, output_dir)


KeyError: 'Date'

In [20]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

# Funzione per caricare i dati
def carica_dati(csv_file):
    """
    Carica i dati dal file CSV e li prepara per la generazione dello spettrogramma.
    """
    dati = pd.read_csv(csv_file)
    return dati

# Funzione per aggiungere rumore bianco gaussiano
def genera_rumore_bianco(seg_data, snr_db=10):
    """
    Aggiunge rumore bianco gaussiano ai dati di chiusura con un rapporto segnale-rumore specificato.
    """
    potenza_segnale = np.mean(seg_data**2)
    potenza_rumore = potenza_segnale / (10**(snr_db / 10))
    rumore_bianco = np.sqrt(potenza_rumore) * np.random.randn(len(seg_data))
    dati_con_rumore = seg_data + rumore_bianco
    return dati_con_rumore

# Funzione per stretching temporale
def time_stretch(seg_data, stretch_factor=1.5):
    """
    Applica stretching temporale ai dati senza cambiare la frequenza.
    """
    return signal.resample(seg_data, int(len(seg_data) * stretch_factor))

# Funzione per pitch shifting
def pitch_shift(seg_data, shift_factor=1.2):
    """
    Modifica la frequenza dei dati spostando il pitch.
    """
    return seg_data * shift_factor

# Funzione per generare spettrogrammi con data augmentation
def genera_spettrogrammi_con_augmentation(dati, anno, num_spettrogrammi, output_dir, snr_db=10):
    """
    Genera spettrogrammi per un dato anno con l'aggiunta di rumore bianco e data augmentation.
    """
    # Filtra i dati per l'anno specificato
    dati_anno = dati[dati['Year'] == anno].iloc[:, 1:].values.flatten()  # Prendi tutte le colonne Day_x
    
    # Dividi i dati in segmenti per generare più spettrogrammi
    segment_length = len(dati_anno) // num_spettrogrammi
    
    for i in range(num_spettrogrammi):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Seleziona i dati del segmento
        seg_data = dati_anno[start_idx:end_idx]
        
        # Aggiungi rumore bianco
        seg_data_rumore = genera_rumore_bianco(seg_data, snr_db)
        
        # Applica data augmentation alternata per creare varietà
        if i % 3 == 0:
            seg_data_rumore = time_stretch(seg_data_rumore, stretch_factor=np.random.uniform(0.8, 1.5))
        elif i % 3 == 1:
            seg_data_rumore = pitch_shift(seg_data_rumore, shift_factor=np.random.uniform(0.8, 1.2))
        else:
            seg_data_rumore = genera_rumore_bianco(seg_data_rumore, snr_db=15)  # Aggiungi ulteriore rumore

        # Genera lo spettrogramma
        f, t, Sxx = signal.spectrogram(seg_data_rumore, nperseg=128)
        
        # Plot dello spettrogramma
        plt.figure(figsize=(5, 4))
        plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [sec]')
        plt.title(f'Spettrogramma - Anno {anno} - {i+1}')
        
        # Salva lo spettrogramma
        file_name = f'spettrogramma_anno_{anno}_{i+1}.png'
        output_path = os.path.join(output_dir, file_name)
        plt.savefig(output_path)
        plt.close()

# Funzione per generare 2000 spettrogrammi in totale
def genera_2000_spettrogrammi(csv_file, output_dir, num_spettrogrammi_totali=2000, snr_db=10):
    """
    Genera spettrogrammi per tutti gli anni nel file CSV, applicando data augmentation.
    """
    # Carica i dati dal file CSV
    dati = carica_dati(csv_file)
    
    # Crea la cartella di output se non esiste
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Trova gli anni unici
    anni = dati['Year'].unique()
    num_anni = len(anni)
    
    # Calcola quanti spettrogrammi generare per ciascun anno
    num_spettrogrammi_per_anno = num_spettrogrammi_totali // num_anni
    
    # Genera gli spettrogrammi per ogni anno
    for anno in anni:
        genera_spettrogrammi_con_augmentation(dati, anno, num_spettrogrammi_per_anno, output_dir, snr_db)

# Esempio di utilizzo
csv_file = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/dataset/adbe.csv'  # Percorso del file CSV con i dati
output_dir = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectrogrammi_adbe'  # Cartella di output per salvare gli spettrogrammi
genera_2000_spettrogrammi(csv_file, output_dir)

/var/folders/jp/fnr369_172b9ghtllhyqdmtw0000gp/T/ipykernel_82985/4046725092.py:70: UserWarning: nperseg = 128 is greater than input length  = 1, using nperseg = 1
  f, t, Sxx = signal.spectrogram(seg_data_rumore, nperseg=128)
/var/folders/jp/fnr369_172b9ghtllhyqdmtw0000gp/T/ipykernel_82985/4046725092.py:74: RuntimeWarning: divide by zero encountered in log10
  plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x111e87b90>>
Traceback (most recent call last):
  File "/Users/roberto/Library/Python/3.12/lib/python/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import gaussian_filter

def carica_dati(csv_file):
    """
    Carica i dati delle serie storiche dal file CSV.
    """
    dati = pd.read_csv(csv_file)
    return dati

def genera_rumore_bianco(dati, snr_db=20):
    """
    Aggiunge rumore bianco gaussiano ai dati delle serie storiche.
    """
    potenza_segnale = np.mean(dati**2)
    potenza_rumore = potenza_segnale / (10**(snr_db / 10))
    rumore_bianco = np.sqrt(potenza_rumore) * np.random.randn(len(dati))
    dati_con_rumore = dati + rumore_bianco
    return dati_con_rumore

def applica_filtro_gaussiano(dati, sigma=1):
    """
    Applica un filtro gaussiano ai dati per creare variazioni.
    """
    return gaussian_filter(dati, sigma=sigma)

def time_stretch(seg_data, stretch_factor=1.5):
    """
    Applica stretching temporale ai dati senza cambiare la frequenza.
    """
    return signal.resample(seg_data, int(len(seg_data) * stretch_factor))

def pitch_shift(seg_data, shift_factor=1.2):
    """
    Modifica la frequenza dei dati spostando il pitch.
    """
    return seg_data * shift_factor

def genera_spettrogramma(dati, anno, output_dir, indice, snr_db=20, apply_filter=False, stretching=False, pitch=False):
    """
    Genera lo spettrogramma per un determinato anno con vari data augmentation.
    """
    # Aggiungi rumore bianco
    dati_rumore = genera_rumore_bianco(dati, snr_db)
    
    # Applica filtri gaussiani
    if apply_filter:
        dati_rumore = applica_filtro_gaussiano(dati_rumore, sigma=np.random.uniform(0.5, 2.0))
    
    # Applica stretching temporale
    if stretching:
        dati_rumore = time_stretch(dati_rumore, stretch_factor=np.random.uniform(0.8, 1.5))
    
    # Applica pitch shifting
    if pitch:
        dati_rumore = pitch_shift(dati_rumore, shift_factor=np.random.uniform(0.8, 1.2))

    # Genera lo spettrogramma
    f, t, Sxx = signal.spectrogram(dati_rumore, nperseg=128)
    
    # Crea la figura del grafico
    plt.figure(figsize=(10, 5))
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
    plt.colorbar(label='dB')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.title(f'Spettrogramma - Anno {anno} - Variante {indice}')

    # Salva lo spettrogramma
    output_file = os.path.join(output_dir, f'spettrogramma_anno_{anno}_variante_{indice}.png')
    plt.savefig(output_file)
    plt.close()

def genera_spettrogrammi_per_anni(csv_file, output_dir, snr_db=20, num_spettrogrammi_totali=2000):
    """
    Genera 2000 spettrogrammi in totale per tutti gli anni del dataset e li salva nella cartella di output.
    """
    # Carica i dati dal CSV
    dati = carica_dati(csv_file)
    
    # Verifica se la cartella di output esiste, altrimenti la crea
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Trova gli anni unici
    anni = dati['Year'].unique()
    num_anni = len(anni)
    
    # Calcola quanti spettrogrammi per anno
    num_spettrogrammi_per_anno = num_spettrogrammi_totali // num_anni
    
    # Cicla su ciascun anno e genera i vari spettrogrammi con data augmentation
    for anno in anni:
        dati_anno = dati[dati['Year'] == anno].iloc[:, 1:].values.flatten()
        
        for i in range(num_spettrogrammi_per_anno):
            # Applica variazioni con diversi tipi di augmentation
            if i % 3 == 0:
                genera_spettrogramma(dati_anno, anno, output_dir, i+1, snr_db=snr_db, apply_filter=True)
            elif i % 3 == 1:
                genera_spettrogramma(dati_anno, anno, output_dir, i+1, snr_db=snr_db, stretching=True)
            else:
                genera_spettrogramma(dati_anno, anno, output_dir, i+1, snr_db=snr_db, pitch=True)

# Esempio di utilizzo
csv_file = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/dataset/adbe.csv'  # Percorso del file CSV
output_dir = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectrogrammi_adbe'  # Cartella di output
genera_spettrogrammi_per_anni(csv_file, output_dir, num_spettrogrammi_totali=2000)


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import gaussian_filter

def carica_dati(csv_file):
    """
    Carica i dati delle serie storiche dal file CSV.
    """
    dati = pd.read_csv(csv_file)
    return dati

def genera_rumore_bianco(dati, snr_db=20):
    """
    Aggiunge rumore bianco gaussiano ai dati delle serie storiche.
    """
    potenza_segnale = np.mean(dati**2)
    potenza_rumore = potenza_segnale / (10**(snr_db / 10))
    rumore_bianco = np.sqrt(potenza_rumore) * np.random.randn(len(dati))
    dati_con_rumore = dati + rumore_bianco
    return dati_con_rumore

def applica_filtro_gaussiano(dati, sigma=1):
    """
    Applica un filtro gaussiano ai dati per creare variazioni.
    """
    return gaussian_filter(dati, sigma=sigma)

def time_stretch(seg_data, stretch_factor=1.5):
    """
    Applica stretching temporale ai dati senza cambiare la frequenza.
    """
    return signal.resample(seg_data, int(len(seg_data) * stretch_factor))

def pitch_shift(seg_data, shift_factor=1.2):
    """
    Modifica la frequenza dei dati spostando il pitch.
    """
    return seg_data * shift_factor

def genera_spettrogramma(dati, anno, output_dir, indice, snr_db=20, apply_filter=False, stretching=False, pitch=False):
    """
    Genera lo spettrogramma per un determinato anno con vari data augmentation.
    """
    # Aggiungi rumore bianco
    dati_rumore = genera_rumore_bianco(dati, snr_db)
    
    # Applica filtri gaussiani
    if apply_filter:
        dati_rumore = applica_filtro_gaussiano(dati_rumore, sigma=np.random.uniform(0.5, 2.0))
    
    # Applica stretching temporale
    if stretching:
        dati_rumore = time_stretch(dati_rumore, stretch_factor=np.random.uniform(0.8, 1.5))
    
    # Applica pitch shifting
    if pitch:
        dati_rumore = pitch_shift(dati_rumore, shift_factor=np.random.uniform(0.8, 1.2))

    # Genera lo spettrogramma
    f, t, Sxx = signal.spectrogram(dati_rumore, nperseg=128)
    
    # Crea la figura del grafico
    plt.figure(figsize=(10, 5))
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
    plt.colorbar(label='dB')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.title(f'Spettrogramma - Anno {anno} - Variante {indice}')

    # Salva lo spettrogramma
    output_file = os.path.join(output_dir, f'spettrogramma_anno_{anno}_variante_{indice}.png')
    plt.savefig(output_file)
    plt.close()

def genera_spettrogrammi_per_anni(csv_file, output_dir, snr_db=20, num_spettrogrammi_totali=2000):
    """
    Genera 2000 spettrogrammi in totale per tutti gli anni del dataset e li salva nella cartella di output.
    """
    # Carica i dati dal CSV
    dati = carica_dati(csv_file)
    
    # Verifica se la cartella di output esiste, altrimenti la crea
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Trova gli anni unici
    anni = dati['Year'].unique()
    num_anni = len(anni)
    
    # Calcola quanti spettrogrammi per anno
    num_spettrogrammi_per_anno = num_spettrogrammi_totali // num_anni
    
    # Cicla su ciascun anno e genera i vari spettrogrammi con data augmentation
    for anno in anni:
        dati_anno = dati[dati['Year'] == anno].iloc[:, 1:].values.flatten()
        
        for i in range(num_spettrogrammi_per_anno):
            # Applica variazioni con diversi tipi di augmentation
            if i % 3 == 0:
                genera_spettrogramma(dati_anno, anno, output_dir, i+1, snr_db=snr_db, apply_filter=True)
            elif i % 3 == 1:
                genera_spettrogramma(dati_anno, anno, output_dir, i+1, snr_db=snr_db, stretching=True)
            else:
                genera_spettrogramma(dati_anno, anno, output_dir, i+1, snr_db=snr_db, pitch=True)

# Esempio di utilizzo
csv_file = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/dataset/jnj.csv'  # Percorso del file CSV
output_dir = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectrogrammi_jnj'  # Cartella di output
genera_spettrogrammi_per_anni(csv_file, output_dir, num_spettrogrammi_totali=2000)


rimuovi contorni


In [4]:
import os
from PIL import Image
import numpy as np

def rimuovi_immagini_vuote(cartella, soglia_varianza=0.01, soglia_bianco=0.95):
    """
    Rimuove tutte le immagini bianche o vuote dalla cartella specificata.
    
    Argomenti:
    - cartella: percorso della cartella dove si trovano le immagini.
    - soglia_varianza: se la varianza dell'immagine è inferiore a questa soglia, l'immagine viene considerata vuota.
    - soglia_bianco: percentuale di pixel bianchi oltre la quale l'immagine viene considerata vuota.
    """
    # Ottieni la lista di file nella cartella
    files = os.listdir(cartella)
    
    # Filtra solo i file immagine (png, jpg, jpeg)
    immagini = [f for f in files if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    for immagine in immagini:
        percorso_immagine = os.path.join(cartella, immagine)
        
        # Carica l'immagine
        try:
            with Image.open(percorso_immagine) as img:
                # Converti l'immagine in scala di grigi
                img_gray = img.convert('L')
                
                # Converti l'immagine in array numpy
                img_array = np.array(img_gray)
                
                # Calcola la varianza dell'immagine
                varianza = np.var(img_array)
                
                # Calcola la percentuale di pixel bianchi (consideriamo "bianchi" i pixel con valore > 245)
                pixel_bianchi = np.sum(img_array > 245)
                percentuale_pixel_bianchi = pixel_bianchi / img_array.size
                
                # Se la varianza è inferiore alla soglia o l'immagine ha troppi pixel bianchi, elimina l'immagine
                if varianza < soglia_varianza or percentuale_pixel_bianchi >= soglia_bianco:
                    os.remove(percorso_immagine)
                    print(f"Rimosso: {immagine} (Varianza: {varianza:.5f}, Pixel bianchi: {percentuale_pixel_bianchi:.2f})")
        
        except Exception as e:
            print(f"Errore nell'elaborazione dell'immagine {immagine}: {e}")

# Esempio di utilizzo
cartella = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectrogrammi_adbe'  # Sostituisci con il percorso della tua cartella
rimuovi_immagini_vuote(cartella)


Rimosso: spettrogramma_anno_2022_variante_74.png (Varianza: 924.76599, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2021_variante_158.png (Varianza: 928.45090, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2019_variante_29.png (Varianza: 921.48151, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2022_variante_146.png (Varianza: 929.83420, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2013_variante_197.png (Varianza: 934.39589, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2022_variante_2.png (Varianza: 927.41201, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2018_variante_8.png (Varianza: 931.79172, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2018_variante_194.png (Varianza: 931.03664, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2021_variante_26.png (Varianza: 928.58128, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2019_variante_137.png (Varianza: 938.73176, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2017_variante_146.png (Varianza: 928.97765, Pixel

In [5]:
import os
from PIL import Image
import numpy as np

def rimuovi_immagini_vuote(cartella, soglia_varianza=0.01, soglia_bianco=0.95):
    """
    Rimuove tutte le immagini bianche o vuote dalla cartella specificata.
    
    Argomenti:
    - cartella: percorso della cartella dove si trovano le immagini.
    - soglia_varianza: se la varianza dell'immagine è inferiore a questa soglia, l'immagine viene considerata vuota.
    - soglia_bianco: percentuale di pixel bianchi oltre la quale l'immagine viene considerata vuota.
    """
    # Ottieni la lista di file nella cartella
    files = os.listdir(cartella)
    
    # Filtra solo i file immagine (png, jpg, jpeg)
    immagini = [f for f in files if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    for immagine in immagini:
        percorso_immagine = os.path.join(cartella, immagine)
        
        # Carica l'immagine
        try:
            with Image.open(percorso_immagine) as img:
                # Converti l'immagine in scala di grigi
                img_gray = img.convert('L')
                
                # Converti l'immagine in array numpy
                img_array = np.array(img_gray)
                
                # Calcola la varianza dell'immagine
                varianza = np.var(img_array)
                
                # Calcola la percentuale di pixel bianchi (consideriamo "bianchi" i pixel con valore > 245)
                pixel_bianchi = np.sum(img_array > 245)
                percentuale_pixel_bianchi = pixel_bianchi / img_array.size
                
                # Se la varianza è inferiore alla soglia o l'immagine ha troppi pixel bianchi, elimina l'immagine
                if varianza < soglia_varianza or percentuale_pixel_bianchi >= soglia_bianco:
                    os.remove(percorso_immagine)
                    print(f"Rimosso: {immagine} (Varianza: {varianza:.5f}, Pixel bianchi: {percentuale_pixel_bianchi:.2f})")
        
        except Exception as e:
            print(f"Errore nell'elaborazione dell'immagine {immagine}: {e}")

# Esempio di utilizzo
cartella = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectrogrammi_jnj'  # Sostituisci con il percorso della tua cartella
rimuovi_immagini_vuote(cartella)


Rimosso: spettrogramma_anno_2019_variante_134.png (Varianza: 932.52911, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2022_variante_74.png (Varianza: 924.76599, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2021_variante_170.png (Varianza: 922.38959, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2020_variante_68.png (Varianza: 933.27338, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2022_variante_152.png (Varianza: 934.75070, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2013_variante_197.png (Varianza: 927.30433, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2019_variante_14.png (Varianza: 932.26910, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2022_variante_2.png (Varianza: 924.18726, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2018_variante_8.png (Varianza: 929.77360, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2015_variante_110.png (Varianza: 929.35082, Pixel bianchi: 0.96)
Rimosso: spettrogramma_anno_2014_variante_158.png (Varianza: 941.60311, Pixel

preprocessing e creazione classe dataset


In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.utils import to_categorical

def carica_spettrogrammi(cartella_tsla, cartella_kk):
    """
    Carica gli spettrogrammi per adbe e jnj, etichettandoli correttamente.
    
    - cartella_tsla: percorso della cartella con gli spettrogrammi di adbe.
    - cartella_kk: percorso della cartella con gli spettrogrammi di jnj.
    """
    spettrogrammi = []
    labels = []
    
    # Carica spettrogrammi TSLA
    for file in os.listdir(cartella_tsla):
        if file.endswith('.png'):
            img = Image.open(os.path.join(cartella_tsla, file)).convert('RGB')
            img_resized = img.resize((128, 128))  # Ridimensiona tutte le immagini
            spettrogrammi.append(np.array(img_resized))
            labels.append(0)  # Etichetta TSLA come 0

    # Carica spettrogrammi KK
    for file in os.listdir(cartella_kk):
        if file.endswith('.png'):
            img = Image.open(os.path.join(cartella_kk, file)).convert('RGB')
            img_resized = img.resize((128, 128))  # Ridimensiona tutte le immagini
            spettrogrammi.append(np.array(img_resized))
            labels.append(1)  # Etichetta KK come 1

    # Converti a numpy array e normalizza
    spettrogrammi = np.array(spettrogrammi) / 255.0
    labels = np.array(labels)
    
    return spettrogrammi, labels

# Carica gli spettrogrammi
spettrogrammi, labels = carica_spettrogrammi('/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi/spectrogrammi_adbe', '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi/spectrogrammi_jnj')

# Suddividi i dati in train (70%), validation (20%) e test (10%)
X_train, X_temp, y_train, y_temp = train_test_split(spettrogrammi, labels, test_size=0.30, stratify=labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, stratify=y_temp, random_state=42)

# Converti le etichette in formato categorico
y_train_cat = to_categorical(y_train, num_classes=2)
y_val_cat = to_categorical(y_val, num_classes=2)
y_test_cat = to_categorical(y_test, num_classes=2)


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def crea_cnn():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  # Due classi: TSLA, KK

    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Creazione del modello CNN
model = crea_cnn()

# Mostra il riassunto del modello
model.summary()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,392,578 (28.20 MB)

 Trainable params: 7,392,578 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [6]:
# Correzione del percorso per il checkpoint del modello
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Definisci callback per l'early stopping e per salvare il miglior modello
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
#checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

# Addestramento del modello
#history = model.fit(X_train, y_train_cat, 
                    #validation_data=(X_val, y_val_cat),
                    #epochs=50, 
                    #batch_size=32, 
                    #callbacks=[early_stop, checkpoint],
                    #verbose=1)


In [11]:
# Definisci un percorso dove hai i permessi di scrittura, ad esempio una cartella sul desktop
checkpoint = ModelCheckpoint('/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectrogrammi/output/final_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

# Addestramento del modello
history = model.fit(X_train, y_train_cat, 
                    validation_data=(X_val, y_val_cat),
                    epochs=50, 
                    batch_size=32, 
                    callbacks=[early_stop, checkpoint],
                    verbose=1)


Epoch 1/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.4877 - loss: 0.6932
Epoch 1: val_loss improved from inf to 0.69313, saving model to /Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectrogrammi/output/final_model.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 8s 104ms/step - accuracy: 0.4879 - loss: 0.6932 - val_accuracy: 0.5027 - val_loss: 0.6931
Epoch 2/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.4913 - loss: 0.6932
Epoch 2: val_loss did not improve from 0.69313
81/81 ━━━━━━━━━━━━━━━━━━━━ 8s 102ms/step - accuracy: 0.4913 - loss: 0.6932 - val_accuracy: 0.5027 - val_loss: 0.6931
Epoch 3/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.5132 - loss: 0.6931
Epoch 3: val_loss did not improve from 0.69313
81/81 ━━━━━━━━━━━━━━━━━━━━ 8s 103ms/step - accuracy: 0.5131 - loss: 0.6931 - val_accuracy: 0.5027 - val_loss: 0.6931
Epoch 4/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.4879 - loss: 0.6934
Epoch 4: val_loss did not improve from 0.69313
81/81 ━━━━━━━━━━━━━━━━━━━

In [12]:
# Valutazione sul test set
test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=1)
print(f'Accuracy sul test set: {test_acc * 100:.2f}%')


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4945 - loss: 0.6932
Accuracy sul test set: 50.14%


In [13]:
import torch
import os
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Verifica se MPS (Metal) o CUDA è disponibile
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Stai utilizzando il dispositivo: {device}")


Stai utilizzando il dispositivo: mps


In [14]:
class SpettrogrammaDataset(Dataset):
    def __init__(self, root_dir, labels, transform=None):
        """
        Args:
            root_dir (str): Directory principale contenente le cartelle con le immagini degli spettrogrammi.
            labels (dict): Dizionario che associa il nome della cartella alla sua etichetta.
            transform (callable, optional): Trasformazioni da applicare alle immagini.
        """
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
        self.images = []
        self.img_labels = []

        for label_name, label in labels.items():
            label_dir = os.path.join(root_dir, label_name)
            for img_file in os.listdir(label_dir):
                if img_file.endswith('.png'):
                    self.images.append(os.path.join(label_dir, img_file))
                    self.img_labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.img_labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 16 * 16, 128)  # Layer fully connected
        self.fc2 = nn.Linear(128, 2)  # Due classi
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [16]:
# Percorso delle cartelle con gli spettrogrammi
root_dir = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi'
labels = {'spectrogrammi_adbe': 0, 'spectrogrammi_jnj': 1}  # Cambia class1 e class2 con i nomi delle tue cartelle

# Trasformazioni e creazione dataset
transform = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor()
])

# Creazione del dataset
dataset = SpettrogrammaDataset(root_dir=root_dir, labels=labels, transform=transform)

# Suddivisione in train, validation, test (70:20:10)
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Creazione dei DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [17]:
def train_model(model, train_loader, val_loader, device, epochs=20, lr=0.001, model_path='best_model.pth'):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model = model.to(device)
    
    best_val_loss = float('inf')  # Inizializza la migliore loss
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        total = 0
        
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            running_acc += torch.sum(preds == labels.data).float()
            total += labels.size(0)
        
        train_loss = running_loss / total
        train_acc = running_acc / total
        
        # Valutazione
        val_loss, val_acc = validate_model(model, val_loader, criterion, device)
        
        # Salva il modello se la validazione è migliorata
        if val_loss < best_val_loss:
            print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving model...")
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_path)
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

def validate_model(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    total = 0
    
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            val_acc += torch.sum(preds == labels.data).float()
            total += labels.size(0)
    
    val_loss = val_loss / total
    val_acc = val_acc / total
    return val_loss, val_acc


In [18]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def test_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    # Matrice di confusione
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)
    
    # Report di classificazione
    report = classification_report(all_labels, all_preds)
    print(report)

    # Visualizzazione della matrice di confusione
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()


In [19]:
# Inizializzazione del modello
model = CNN()

# Avvio dell'addestramento
train_model(model, train_loader, val_loader, device, epochs=50, lr=0.001, model_path='/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectrogrammi/output/best_model.pth')

# Carica il miglior modello salvato
model.load_state_dict(torch.load('/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectrogrammi/output/best_model.pth'))

# Testa il modello sui dati di test
test_model(model, test_loader, device)


Validation loss improved from inf to 0.6928. Saving model...
Epoch 1/50, Train Loss: 0.6990, Train Acc: 0.4809, Val Loss: 0.6928, Val Acc: 0.5284
Epoch 2/50, Train Loss: 0.6937, Train Acc: 0.5010, Val Loss: 0.6952, Val Acc: 0.4716
Epoch 3/50, Train Loss: 0.6948, Train Acc: 0.5041, Val Loss: 0.6934, Val Acc: 0.4716
Epoch 4/50, Train Loss: 0.6931, Train Acc: 0.5133, Val Loss: 0.6930, Val Acc: 0.5284
Epoch 5/50, Train Loss: 0.6940, Train Acc: 0.5172, Val Loss: 0.6934, Val Acc: 0.4716
Epoch 6/50, Train Loss: 0.6932, Train Acc: 0.5125, Val Loss: 0.6946, Val Acc: 0.4716
Epoch 7/50, Train Loss: 0.6929, Train Acc: 0.5137, Val Loss: 0.6946, Val Acc: 0.4716
Epoch 8/50, Train Loss: 0.6929, Train Acc: 0.5137, Val Loss: 0.6949, Val Acc: 0.4716
Epoch 9/50, Train Loss: 0.6930, Train Acc: 0.5137, Val Loss: 0.6948, Val Acc: 0.4716
Epoch 10/50, Train Loss: 0.6929, Train Acc: 0.5137, Val Loss: 0.6950, Val Acc: 0.4716
Epoch 11/50, Train Loss: 0.6929, Train Acc: 0.5137, Val Loss: 0.6950, Val Acc: 0.4716
Ep

KeyboardInterrupt: 

In [20]:
import torch
import os
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Verifica se MPS (Metal) o CUDA è disponibile
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Stai utilizzando il dispositivo: {device}")


Stai utilizzando il dispositivo: mps


In [21]:
class SpettrogrammaDataset(Dataset):
    def __init__(self, root_dir, labels, transform=None):
        """
        Args:
            root_dir (str): Directory principale contenente le cartelle con le immagini degli spettrogrammi.
            labels (dict): Dizionario che associa il nome della cartella alla sua etichetta.
            transform (callable, optional): Trasformazioni da applicare alle immagini.
        """
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
        self.images = []
        self.img_labels = []

        for label_name, label in labels.items():
            label_dir = os.path.join(root_dir, label_name)
            for img_file in os.listdir(label_dir):
                if img_file.endswith('.png'):
                    self.images.append(os.path.join(label_dir, img_file))
                    self.img_labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.img_labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [22]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 16 * 16, 128)  # Flatten layer per immagini 128x128
        self.fc2 = nn.Linear(128, 2)  # Due classi: JNJ e ADBE
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [23]:
# Percorso delle cartelle con gli spettrogrammi
root_dir = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi'  # Inserisci il percorso corretto
labels = {'spectrogrammi_jnj': 0, 'spectrogrammi_adbe': 1}  # Etichette delle cartelle

# Trasformazioni per le immagini (es. ridimensionamento e conversione in tensori)
transform = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor()
])

# Creazione del dataset
dataset = SpettrogrammaDataset(root_dir=root_dir, labels=labels, transform=transform)

# Suddivisione in train (70%), validation (20%), test (10%)
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Creazione dei DataLoader per batch di immagini
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [24]:
def train_model(model, train_loader, val_loader, device, epochs=20, lr=0.001, model_path='best_model.pth'):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model = model.to(device)
    
    best_val_loss = float('inf')  # Inizializza la migliore loss
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        total = 0
        
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            running_acc += torch.sum(preds == labels.data).float()
            total += labels.size(0)
        
        train_loss = running_loss / total
        train_acc = running_acc / total
        
        # Valutazione sul validation set
        val_loss, val_acc = validate_model(model, val_loader, criterion, device)
        
        # Salva il modello se la validazione è migliorata
        if val_loss < best_val_loss:
            print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving model...")
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_path)
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')


In [25]:
def validate_model(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    total = 0
    
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            val_acc += torch.sum(preds == labels.data).float()
            total += labels.size(0)
    
    val_loss = val_loss / total
    val_acc = val_acc / total
    return val_loss, val_acc


In [27]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def test_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    # Matrice di confusione
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # Report di classificazione
    report = classification_report(all_labels, all_preds, target_names=['JNJ', 'ADBE'])
    print("Classification Report:")
    print(report)

    # Visualizzazione della matrice di confusione
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['JNJ', 'ADBE'], yticklabels=['JNJ', 'ADBE'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()




In [28]:
# Inizializzazione del modello
model = CNN()

# Addestramento del modello con 50 epoche
train_model(model, train_loader, val_loader, device, epochs=20, lr=0.001, model_path='/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectrogrammi/output/best_model.pth')

# Caricamento del miglior modello salvato
model.load_state_dict(torch.load('/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spectrogrammi/output/best_model.pth'))

# Test del modello sui dati di test
test_model(model, test_loader, device)


Validation loss improved from inf to 0.6936. Saving model...
Epoch 1/20, Train Loss: 0.6953, Train Acc: 0.5017, Val Loss: 0.6936, Val Acc: 0.4892
Epoch 2/20, Train Loss: 0.6935, Train Acc: 0.5083, Val Loss: 0.6936, Val Acc: 0.4892
Epoch 3/20, Train Loss: 0.6931, Train Acc: 0.5075, Val Loss: 0.6936, Val Acc: 0.4892
Epoch 4/20, Train Loss: 0.6931, Train Acc: 0.5075, Val Loss: 0.6936, Val Acc: 0.4892
Epoch 5/20, Train Loss: 0.6932, Train Acc: 0.5075, Val Loss: 0.6936, Val Acc: 0.4892
Validation loss improved from 0.6936 to 0.6934. Saving model...
Epoch 6/20, Train Loss: 0.6932, Train Acc: 0.5010, Val Loss: 0.6934, Val Acc: 0.4892


KeyboardInterrupt: 